In [1]:
# import sys
# import os
# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

In [2]:
import numpy as np

from my_stuff import MufexKeys
import requests
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.enums import DynamicOrderSettingsArrays
from quantfreedom.enums import *
from quantfreedom.helper_funcs import dos_cart_product
from quantfreedom.nb_funcs.nb_strategies.nb_strategy import (
    nb_strat_bt_create_ind,
    nb_strat_evaluate,
    nb_strat_get_current_ind_settings,
    nb_strat_get_ind_set_str,
    nb_strat_get_total_ind_settings,
)
from quantfreedom.nb_funcs.nb_simulate import nb_sim_df_backtest
from quantfreedom.sim_order_records import sim_get_or
from quantfreedom.nb_funcs.nb_sim_order_records import nb_sim_get_or


mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2

In [3]:
candles_np = mufex_main.get_candles(
    symbol="BTCUSDT", timeframe="5m", candles_to_dl=3000)

It took 00 mins and 01 seconds to download 3000 candles


In [4]:
mufex_main.set_exchange_settings(
    symbol="BTCUSDT",
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)
backtest_settings = BacktestSettings(
    divide_records_array_size_by=1,
    gains_pct_filter=-50,
    total_trade_filter=40,
    upside_filter=-np.inf,
)
dos_arrays = DynamicOrderSettingsArrays(
    entry_size_asset=np.array([0]),
    max_equity_risk_pct=np.array([6, 9, 12]),
    max_trades=np.array([10]),
    num_candles=np.array([100]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([2, 4, 6]),
    sl_based_on_add_pct=np.array([0.1, 0.5, 1.0, 0.05]),
    sl_based_on_lookback=np.array([10, 30]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    sl_to_be_ze_type=np.array([0]),
    static_leverage=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.1, 0.5, 1.0, 2]),
    trail_sl_when_pct=np.array([0.2, 0.5, 1, 2]),
)
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    long_or_short=LongOrShortType.Long,
    logger_bool=True,
    pg_min_max_sl_bcb=PriceGetterType.Min,
    sl_to_be_bool=False,
    z_or_e_type=ZeroOrEntryType.Nothing,
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    tp_fee_type=TakeProfitFeeType.Limit,
    trail_sl_bool=True,
)
dos_cart_arrays = dos_cart_product(
    dos_arrays=dos_arrays,
)

ExchangeSettings(
    limit_fee_pct = 0.0003,
    max_leverage = 150.0,
    market_fee_pct = 0.0009,
    mmr_pct = 0.004,
    min_leverage = 1.0,
    max_asset_size = 100.0,
    min_asset_size = 0.001,
    asset_tick_step = 3,
    position_mode = 1,
    leverage_mode = 1,
    price_tick_step = 1,
    leverage_tick_step = 2,
)


In [5]:
strat_records_df = nb_sim_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles_np,
    dos_cart_arrays=dos_cart_arrays,
    evaluate=nb_strat_evaluate,
    exchange_settings=mufex_main.exchange_settings,
    get_current_ind_settings=nb_strat_get_current_ind_settings,
    get_ind_set_str=nb_strat_get_ind_set_str,
    get_total_ind_settings=nb_strat_get_total_ind_settings,
    ind_creator=nb_strat_bt_create_ind,
    nb_logger_type=NBLoggerType.Pass,
    starting_equity=1000.0,
    static_os=static_os,
)
strat_records_df

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 6
Total order settings to test: 1,152
Total combinations of settings to test: 6,912
Total candles: 3,000
Total candles to test: 20,736,000


,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,to_the_upside,fees_paid,total_pnl,ending_eq
0,0,2,65.0,26,39,-2.04,40.00,-0.0000,1126.190,-20.4113,979.5887
1,0,3,45.0,19,26,32.82,42.22,0.0000,743.614,328.1553,1328.1553
2,0,5,113.0,39,74,-29.68,34.51,-0.0000,1851.853,-296.8197,703.1803
3,0,6,89.0,32,57,22.57,35.96,0.0000,1693.793,225.6883,1225.6883
4,0,7,41.0,19,22,86.07,46.34,0.0000,774.829,860.7186,1860.7186
...,...,...,...,...,...,...,...,...,...,...,...
1264,3,1132,41.0,6,35,-20.43,14.63,-0.0000,1512.725,-204.3165,795.6835
1265,3,1133,41.0,6,35,-20.43,14.63,-0.0000,1512.725,-204.3165,795.6835
1266,3,1134,41.0,6,35,-20.43,14.63,-0.0000,1512.725,-204.3165,795.6835
1267,3,1144,60.0,13,47,-46.25,21.67,-0.3336,2316.071,-462.5465,537.4535


In [6]:
strat_records_df.sort_values(by='to_the_upside', ascending=False)

,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,to_the_upside,fees_paid,total_pnl,ending_eq
271,0,848,44.0,33,11,58.38,75.00,0.6084,581.827,583.8367,1583.8367
457,1,321,42.0,33,9,47.32,78.57,0.6033,285.984,473.2421,1473.2421
114,0,321,44.0,34,10,47.48,77.27,0.5757,318.915,474.7798,1474.7798
10,0,13,52.0,21,31,69.92,40.38,0.5479,1079.895,699.2296,1699.2296
9,0,12,52.0,21,31,69.92,40.38,0.5479,1079.895,699.2296,1699.2296
...,...,...,...,...,...,...,...,...,...,...,...
1191,3,688,62.0,33,29,-44.25,53.23,-0.9265,590.739,-442.5457,557.4543
1162,3,560,62.0,33,29,-44.25,53.23,-0.9265,590.739,-442.5457,557.4543
1131,3,432,62.0,33,29,-44.25,53.23,-0.9265,590.739,-442.5457,557.4543
742,2,161,70.0,39,31,-39.70,55.71,-0.9278,603.003,-397.0151,602.9849


In [11]:
order_records_df, data = nb_sim_get_or(
    backtest_settings=backtest_settings,
    candles=candles_np,
    dos_cart_arrays=dos_cart_arrays,
    evaluate=nb_strat_evaluate,
    exchange_settings=mufex_main.exchange_settings,
    get_current_ind_settings=nb_strat_get_current_ind_settings,
    get_ind_set_str=nb_strat_get_ind_set_str,
    get_total_ind_settings=nb_strat_get_total_ind_settings,
    ind_creator=nb_strat_bt_create_ind,
    nb_logger_type=NBLoggerType.Pass,
    starting_equity=1000.0,
    static_os=static_os,
    ind_set_index=0,
    dos_index=848,
)

In [12]:
order_records_df

,ind_set_idx,or_set_idx,bar_idx,timestamp,datetime,order_status,equity,available_balance,cash_borrowed,cash_used,...,entry_size_usd,entry_price,exit_price,position_size_asset,position_size_usd,realized_pnl,sl_pct,sl_price,tp_pct,tp_price
0,0,848,100,1697713200000,2023-10-19 11:00:00,EntryFilled,1000.0000,956.951,1585.713,43.049,...,1628.762,28460.0,NaN,0.057,1628.762,NaN,1.66,27986.6,3.8,29542.9
1,0,848,101,1697713500000,2023-10-19 11:05:00,EntryFilled,1000.0000,912.009,3220.201,87.991,...,1722.479,28431.0,NaN,0.118,3351.241,NaN,1.63,27986.6,3.7,29503.3
2,0,848,102,1697713800000,2023-10-19 11:10:00,EntryFilled,1000.0000,868.536,4763.258,131.464,...,1674.521,28435.4,NaN,0.177,5025.762,NaN,1.62,27986.6,3.7,29499.7
3,0,848,103,1697714100000,2023-10-19 11:15:00,EntryFilled,1000.0000,823.556,6330.944,176.444,...,1744.130,28421.3,NaN,0.238,6769.892,NaN,1.60,27986.6,3.7,29484.0
4,0,848,129,1697721900000,2023-10-19 13:25:00,MovedTSL,1000.0000,823.556,6330.944,176.444,...,1744.130,28421.3,NaN,0.238,6769.892,NaN,0.13,28477.6,3.7,29484.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,0,848,2993,1698581100000,2023-10-29 12:05:00,MovedTSL,1583.8367,1364.361,8700.363,219.475,...,3138.464,34188.7,NaN,0.267,9136.294,NaN,0.14,34257.6,3.3,35328.1
358,0,848,2994,1698581400000,2023-10-29 12:10:00,MovedTSL,1583.8367,1364.361,8700.363,219.475,...,3138.464,34188.7,NaN,0.267,9136.294,NaN,0.17,34266.6,3.3,35328.1
359,0,848,2996,1698582000000,2023-10-29 12:20:00,MovedTSL,1583.8367,1364.361,8700.363,219.475,...,3138.464,34188.7,NaN,0.267,9136.294,NaN,0.21,34279.7,3.3,35328.1
360,0,848,2998,1698582600000,2023-10-29 12:30:00,MovedTSL,1583.8367,1364.361,8700.363,219.475,...,3138.464,34188.7,NaN,0.267,9136.294,NaN,0.23,34286.4,3.3,35328.1


In [13]:
res = requests.post("http://127.0.0.1:7779/plot", json=data)

returned_data = res.json()

print(returned_data)

{'ok': True}
